In [48]:
import pandas as pd
import re
from functools import reduce

In [11]:
dataUrls= ["http://schools.nyc.gov/NR/rdonlyres/19E79FA9-ED99-42BA-B89E-56CB931F2465/0/2014_2015_HS_SQR_Results_2016_04_08.xlsx",
          "http://schools.nyc.gov/NR/rdonlyres/32595FE4-15E0-4DFE-A4F4-2D9AD32ED6D1/0/2015_2016_HS_SQR_Results_2017_01_05.xlsx",
          "http://schools.nyc.gov/NR/rdonlyres/CDB6D687-3DAB-4B0D-8DC7-0EF7784E7C3B/0/201617_HS_SQR_Results_2018_01_22.xlsx"]
dataSheets = ["Summary","Student Achievement","Closing the Achievement Gap"]
summaryColsRegex = "(?<!Trust)Percent (?!Positive|of)|(8)|Economic|Name|Type|Enrollment"
achievementCols = ['School Name','Metric Value - Graduation Rate, 4 year',
                  'Metric Value - Four-Year College Readiness Index']
closingCols = ['School Name',
               'Metric Value - Graduation Rate, 4-year, black/hispanic lowest third city',
              'Metric Value - Graduation Rate, 4-year, Self-Contained, CTT, or SETSS',
              'Metric Value - Graduation Rate, 4-year, ELL']

In [16]:
temp = pd.read_excel(io=url,sheet_name = dataSheets,header=1)
temp["Summary"] = temp["Summary"].filter(regex = summaryColsRegex)
temp["Student Achievement"] = temp["Student Achievement"].filter(achievementCols)
temp["Closing the Achievement Gap"]
#full = pd.concat(objs=temp,axis = 1)
year = re.search("\d{4}_\d{4}",url)[0]
full['School Year'] = year